# Support Vector Regression (SVR) Hands-on Exercise

In this exercise, we will implement and train a Support Vector Regression model using JAX and evaluate its performance on synthetic data. We will go through the steps of data generation, model training, and evaluation.


### Step 1: Import Libraries

We will begin by importing the necessary libraries. JAX will be used for numerical computations, NumPy for array manipulations, and Matplotlib for visualization.


In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Step 2: Define the SVR Class

We will create a class `SVR` to encapsulate the SVR model. The class will include methods for calculating the loss, training the model, and making predictions.

#### Loss Function

The loss function for SVR is defined as (soft-margin):

$$ L(w) = \lambda \|w\|^2 + \frac{1}{n} \sum_{i=1}^{n} \max(0, |f(x_i) - y_i| - \epsilon) $$

Where:

- $ w $ are the model parameters.
- $ \lambda $ is the regularization parameter.
- $ \epsilon $ is the epsilon-insensitive loss threshold.
- $ f(x_i) $ is the predicted value for input $ x_i $.
- $ y_i $ is the actual target value.

Let's define the `SVR` class.


In [ ]:
class SVR:
    def __init__(self, epsilon=0.1, lmbda=1.0):
        self.epsilon = epsilon
        self.lmbda = lmbda
        self.w = None

    def loss(self, params, X, y):
        # SOLUTION-BEGIN
        predictions = X.reshape((-1, self.n_features)) @ params[:-1] + params[-1]

        # Compute epsilon-insensitive loss
        epsilon_loss = jnp.maximum(0, jnp.abs(predictions - y) - self.epsilon)

        # Regularization term (L2 norm of w)
        reg_term = self.lmbda * jnp.sum(params**2)

        # Total loss
        return reg_term + jnp.mean(epsilon_loss)
        # SOLUTION-END

    def train(self, X, y, lr=1e-2, max_iter=1000):
        # SOLUTION-BEGIN
        _, self.n_features = X.shape

        # Initialize weights and bias
        self.w = jnp.zeros(self.n_features + 1)

        # Solve optimization problem
        grad_fn = jax.grad(self.loss, argnums=0)

        @jax.jit
        def step(w):
            return w - lr * grad_fn(w, X, y)

        for _ in range(max_iter):
            self.w = step(self.w)
        # SOLUTION-END

    def predict(self, X):
        # SOLUTION-BEGIN
        return X.reshape((-1, self.n_features)) @ self.w[:-1] + self.w[-1]
        # SOLUTION-END

### Step 3: Generate Synthetic Data

Next, we will generate synthetic data for regression. We will create a linear relationship with added Gaussian noise.

- We will generate $ n\_{samples} $ data points.
- The true relationship will follow the equation $ y = mx + c $.
- Gaussian noise will be added to simulate real-world data.


In [ ]:
np.random.seed(0)  # For reproducibility
m = 2.5  # Slope
c = 1.0  # Intercept
n_samples = 100

# Generate X values uniformly distributed between 0 and 10
X = np.random.uniform(0, 10, size=(n_samples, 1))

# Create the line (y = mx + c) and add Gaussian noise
noise = np.random.normal(0, 1, size=(n_samples, 1))  # Gaussian noise
y = m * X + c + noise  # Add noise to the true line
y = y.flatten()

### Step 4: Split the Data

Now, we will split the dataset into training and testing sets. This allows us to evaluate the model's performance on unseen data.

We will use an 80-20 split for training and testing.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = jnp.array(X_train)
X_test = jnp.array(X_test)
y_train = jnp.array(y_train)
y_test = jnp.array(y_test)

### Step 5: Train the SVR Model

We will now create an instance of the `SVR` class and train the model using the training data.


In [ ]:
svr = SVR(epsilon=1.0, lmbda=0.1)
svr.train(X_train, y_train)

### Step 6: Make Predictions

After training, we will use the model to make predictions on both the training and testing datasets.


In [ ]:
y_pred_train = svr.predict(X_train)
y_pred_test = svr.predict(X_test)

### Step 7: Evaluate the Model

We will evaluate the model's performance using the Mean Squared Error (MSE), which is defined as:

$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

Where:

- $ y_i $ is the actual value.
- $ \hat{y}_i $ is the predicted value.
- $ n $ is the number of samples.


In [ ]:
mse_train = jnp.mean((y_train - y_pred_train) ** 2)
mse_test = jnp.mean((y_test - y_pred_test) ** 2)
print(f"Train MSE: {mse_train:.4f}")
print(f"Test MSE: {mse_test:.4f}")

### Step 8: Visualize the Results

Finally, we will visualize the training data, test data, and the model's predictions. This will help us understand how well our model has captured the underlying relationship.


In [ ]:
plt.figure(figsize=(10, 6))

# Plot training data
plt.scatter(X_train, y_train, color="blue", label="Training data")
# Plot test data
plt.scatter(X_test, y_test, color="green", marker="x", label="Test data")

# Plot the prediction line (SVR model)
x_range = jnp.linspace(0, 10, 100)
y_pred_line = svr.predict(x_range)
plt.plot(x_range, y_pred_line, color="red", label="SVR prediction")
plt.fill_between(
    x_range,
    y_pred_line - svr.epsilon,
    y_pred_line + svr.epsilon,
    label="SVR tube",
    color="r",
    alpha=0.1,
)

plt.xlabel("X")
plt.ylabel("y")
plt.title("SVR on Synthetic Data with Gaussian Noise")
plt.legend()
plt.show()

# Support Vector Machine (SVM)

We will change the previous code to implement a linear Support Vector Machine in primal formulation


### Step 2: Define the SVM Class

We will create a class `SVM` to encapsulate the SVM model. The class will include methods for calculating the loss, training the model, and making predictions.

### Loss Function

The loss function for SVM is defined as the hinge loss:

$$ L(w) = \lambda \|w\|^2 + \frac{1}{n} \sum_{i=1}^{n} \max(0, 1 - y_i (w^T x_i + b)) $$

Where:

- $ w $ are the model parameters (weights).
- $ b $ is the bias term.
- $ \lambda $ is the regularization parameter.
- $ y_i $ is the true label for the sample $ i $.
- $ x_i $ is the feature vector for the sample $ i $.
- The first term is the regularization term, and the second term is the hinge loss.

Let's define the `SVM` class.


In [ ]:
class SVM:
    def __init__(self, lmbda=1.0):
        self.lmbda = lmbda
        self.w = None

    def loss(self, params, X, y):
        # SOLUTION-BEGIN
        # Compute the decision function
        print(X.shape, params.shape, y.shape)
        decision = X @ params[:-1] + params[-1]
        # Compute the hinge loss
        loss_val = jnp.maximum(0, 1 - y * decision)
        # Regularization term (L2 norm of w)
        reg_term = self.lmbda * jnp.sum(params**2)
        # Total loss
        return reg_term + jnp.mean(loss_val)
        # SOLUTION-END

    def train(self, X, y, lr=1e-2, max_iter=1000):
        # SOLUTION-BEGIN
        _, self.n_features = X.shape

        # Initialize weights and bias
        self.w = jnp.zeros(self.n_features + 1)

        # Solve optimization problem
        grad_fn = jax.grad(self.loss, argnums=0)

        @jax.jit
        def step(w):
            return w - lr * grad_fn(w, X, y)

        for _ in range(max_iter):
            self.w = step(self.w)
        # SOLUTION-END

    def predict(self, X):
        # SOLUTION-BEGIN
        # Decision function
        decision = X @ self.w[:-1] + self.w[-1]
        return jnp.sign(decision)
        # SOLUTION-END

### Step 3: Generate Synthetic Classification Data

Next, we will generate synthetic data for classification. We will create a dataset of points in a 2D space and label them based on their coordinates.

- We will generate $n_{samples} $ data points.
- The labels will be determined by the condition $ x_1 + x_2 > 10 $ to classify points into two categories.


In [ ]:
np.random.seed(42)  # For reproducibility
n_samples = 100

# Generate X values uniformly distributed between [0, 10]^2
X = np.random.uniform(0, 10, size=(n_samples, 2))

# Generate binary labels
y = np.where(X.sum(axis=1) > 10, 1, -1)
y = y.flatten()

### Step 4: Split the Data

Now, we will split the dataset into training and testing sets. This allows us to evaluate the model's performance on unseen data.

We will use an 80-20 split for training and testing.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = jnp.array(X_train)
X_test = jnp.array(X_test)
y_train = jnp.array(y_train)
y_test = jnp.array(y_test)

### Step 5: Train the SVM Model

We will now create an instance of the `SVM` class and train the model using the training data.


In [ ]:
svm = SVM(lmbda=0.001)
svm.train(X_train, y_train, lr=1e-1, max_iter=5000)
print("Loss: ", svm.loss(svm.w, X_train, y_train))
print(svm.w)

### Step 6: Make Predictions

After training, we will use the model to make predictions on both the training and testing datasets.


In [ ]:
y_pred_train = svm.predict(X_train)
y_pred_test = svm.predict(X_test)

### Step 7: Evaluate the Model

We will evaluate the model's performance using accuracy, which is defined as the proportion of correct predictions:

$$ \text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}} $$

Let's calculate and print the accuracy for both the training and test sets.


In [ ]:
accuracy_train = jnp.mean(y_pred_train == y_train)
accuracy_test = jnp.mean(y_pred_test == y_test)
print(f"Train Accuracy: {accuracy_train:.4f}")
print(f"Test Accuracy: {accuracy_test:.4f}")

### Step 8: Visualize the Results

Finally, we will visualize the training data, test data, and the decision boundary of the SVM model. This will help us understand how well our model has separated the classes.


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, label="Training data", marker="o")
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, label="Test data", marker="x")

# Plot the decision boundary
t = jnp.linspace(0, 10, 1000)
xx1, xx2 = jnp.meshgrid(t, t)
xx = jnp.stack([xx1.flatten(), xx2.flatten()], axis=1)
yy = svm.predict(xx)
plt.contourf(xx1, xx2, yy.reshape(xx1.shape), alpha=0.1)
plt.xlabel("X1")
plt.ylabel("X2")
plt.title("SVM Decision Boundary on Synthetic Data")
plt.legend(loc="upper right")
plt.show()